<a href="https://colab.research.google.com/github/huricane85/FPL-API-Fixture-modeler/blob/main/FPL_API_%2B_modeller_%2B_viz_v2_28_10_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://gist.github.com/calumrussell/9a869765af0698fd4fa934ca90029fc8

In [ ]:
! git clone https://github.com/huricane85/FPL-API-Fixture-modeler

Cloning into 'FPL-API-Fixture-modeler'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 21 (delta 9), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [ ]:
from urllib.request import urlopen
from json import loads
from csv import writer


def team_converter(team_id):
    '''Converts a team's ID to their actual name'''
    team_map = {
        1: "Arsenal",
        2: "Aston Villa",
        3: "Brighton",
        4: "Burnley",
        5: "Chelsea",
        6: "Crystal Palace",
        7: "Everton",
        8: "Fulham",
        9: "Leicester City",
        10: "Leeds Utd",
        11: "Liverpool",
        12: "Manchester City",
        13: "Manchester Utd",
        14: "Newcastle utd",
        15: "Sheffield Utd",
        16: "Southampton",
        17: "Tottenham",
        18: "West Brom",
        19: "West Ham Utd",
        20: "Wolverhampton",
        None: None
    }
    return team_map[team_id]


def position_converter(position):
    '''Converts a player's element_type to their actual position'''
    position_map = {
        1: "Goalkeeper",
        2: "Defender",
        3: "Midfielder",
        4: "Forward"
    }
    return position_map[position]


def main():
    all_data = loads(urlopen("https://fantasy.premierleague.com/api/bootstrap-static/").read())
    players = all_data["elements"]

    important_data = [
        [
            x["id"],
            team_converter(x["team"]),
            position_converter(x["element_type"])[0],
            x["web_name"],
            x["now_cost"] / 10,
            x["selected_by_percent"],
            x["goals_scored"],
            x["assists"],
            x["bps"],
            x["ict_index"],
            x["total_points"],  
            x["points_per_game"]
        ]
        for x in players
    ]

    with open("players_data.csv", "w", encoding="utf-8", newline="") as out:
        headers = ["id","team", "pos", "name", "cost", "selected_by_percent", "goals_scored", "assists", "bps", "ict_index","points", "points_per_game"]
        w = writer(out)
        w.writerow(headers)
        w.writerows(important_data)

if __name__ == "__main__":
    main()




In [ ]:
import pandas as pd
playerdata = pd.read_csv("/content/players_data.csv")
playerdata

,id,team,pos,name,cost,selected_by_percent,goals_scored,assists,bps,ict_index,points,points_per_game
0,1,Arsenal,M,Özil,6.8,0.7,0,0,0,0.0,0,0.0
1,2,Arsenal,D,Sokratis,4.9,0.1,0,0,0,0.0,0,0.0
2,3,Arsenal,D,David Luiz,5.5,1.1,0,0,51,9.7,7,1.4
3,4,Arsenal,M,Aubameyang,11.7,15.0,1,1,73,31.4,20,3.3
4,5,Arsenal,D,Cédric,4.7,0.2,0,0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
585,545,Wolverhampton,D,Hoever,4.5,0.0,0,0,9,3.1,2,1.0
586,546,Wolverhampton,D,Semedo,5.5,2.3,0,0,72,12.4,15,3.8
587,553,Wolverhampton,M,Shabani,4.5,0.0,0,0,0,0.0,0,0.0
588,563,Wolverhampton,D,Ait Nouri,5.0,0.0,0,0,0,0.0,0,0.0


In [ ]:
import pandas as pd
import scipy
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

In [ ]:
teamadjustments = pd.read_csv("/content/FPL-API-Fixture-modeler/Predicted_match_scores_GW6-10.csv")
teamadjustments
teamadjustments.columns

Index(['team', 'GW6 Predicted goals ', 'GW6 Average Predicted Goals',
       'GW6 Difference verus average', 'GW7 Predicted goals ',
       'GW7 Average Predicted Goals', 'GW7 Difference verus average',
       'GW8Predicted goals ', 'GW8 Average Predicted Goals',
       'GW8 Difference verus average', 'GW9 Predicted goals ',
       'GW9 Average Predicted Goals', 'GW9 Difference verus average',
       'GW10 Predicted goals ', 'GW10 Average Predicted Goals',
       'GW10 Difference verus average', 'Unnamed: 16', 'Average over 5 GWs'],
      dtype='object')

In [ ]:
playersteamadjusted = pd.merge(playerdata,
                 teamadjustments[['team','GW6 Difference verus average','GW7 Difference verus average','GW8 Difference verus average','GW9 Difference verus average','GW10 Difference verus average']],
                    on='team')
playersteamadjusted

,id,team,pos,name,cost,selected_by_percent,goals_scored,assists,bps,ict_index,points,points_per_game,GW6 Difference verus average,GW7 Difference verus average,GW8 Difference verus average,GW9 Difference verus average,GW10 Difference verus average
0,1,Arsenal,M,Özil,6.8,0.7,0,0,0,0.0,0,0.0,1.08,0.91,1.65,1.18,0.0
1,2,Arsenal,D,Sokratis,4.9,0.1,0,0,0,0.0,0,0.0,1.08,0.91,1.65,1.18,0.0
2,3,Arsenal,D,David Luiz,5.5,1.1,0,0,51,9.7,7,1.4,1.08,0.91,1.65,1.18,0.0
3,4,Arsenal,M,Aubameyang,11.7,15.1,1,1,73,31.4,20,3.3,1.08,0.91,1.65,1.18,0.0
4,5,Arsenal,D,Cédric,4.7,0.2,0,0,0,0.0,0,0.0,1.08,0.91,1.65,1.18,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554,545,Wolverhampton,D,Hoever,4.5,0.0,0,0,9,3.1,2,1.0,0.85,0.98,0.67,0.93,0.0
555,546,Wolverhampton,D,Semedo,5.5,2.3,0,0,72,12.4,15,3.8,0.85,0.98,0.67,0.93,0.0
556,553,Wolverhampton,M,Shabani,4.5,0.0,0,0,0,0.0,0,0.0,0.85,0.98,0.67,0.93,0.0
557,563,Wolverhampton,D,Ait Nouri,5.0,0.0,0,0,0,0.0,0,0.0,0.85,0.98,0.67,0.93,0.0


In [ ]:
playersteamadjusted['GW6 Adjusted score'] = playersteamadjusted['points_per_game'] * playersteamadjusted['GW6 Difference verus average']
playersteamadjusted['GW7 Adjusted score'] = playersteamadjusted['points_per_game'] * playersteamadjusted['GW7 Difference verus average']
playersteamadjusted['GW8 Adjusted score'] = playersteamadjusted['points_per_game'] * playersteamadjusted['GW8 Difference verus average']
playersteamadjusted['GW9 Adjusted score'] = playersteamadjusted['points_per_game'] * playersteamadjusted['GW9 Difference verus average']
playersteamadjusted['GW10 Adjusted score'] = playersteamadjusted['points_per_game'] * playersteamadjusted['GW10 Difference verus average']
playersteamadjusted['GW6-10 Adjusted score'] = playersteamadjusted['GW6 Adjusted score'] + playersteamadjusted['GW7 Adjusted score'] + playersteamadjusted['GW8 Adjusted score'] + playersteamadjusted['GW9 Adjusted score']+ playersteamadjusted['GW10 Adjusted score']
GWprojections = playersteamadjusted

In [ ]:
GWprojections.sort_values(by=['GW6-10 Adjusted score'], ascending=False)

,id,team,pos,name,cost,selected_by_percent,goals_scored,assists,bps,ict_index,points,points_per_game,GW6 Difference verus average,GW7 Difference verus average,GW8 Difference verus average,GW9 Difference verus average,GW10 Difference verus average,GW6 Adjusted score,GW7 Adjusted score,GW8 Adjusted score,GW9 Adjusted score,GW10 Adjusted score,GW6-10 Adjusted score
449,390,Tottenham,M,Son,9.5,56.0,8,2,227,86.1,69,11.5,1.22,1.19,1.98,0.66,0.84,14.030,13.685,22.770,7.590,9.660,67.735
447,388,Tottenham,F,Kane,10.9,41.1,5,8,245,89.7,65,10.8,1.22,1.19,1.98,0.66,0.84,13.176,12.852,21.384,7.128,9.072,63.612
311,254,Liverpool,M,Salah,12.4,47.6,6,1,177,96.0,50,8.3,1.43,1.82,0.76,1.61,1.64,11.869,15.106,6.308,13.363,13.612,60.258
308,251,Liverpool,M,Mané,12.0,9.5,4,2,124,64.5,39,7.8,1.43,1.82,0.76,1.61,1.64,11.154,14.196,5.928,12.558,12.792,56.628
367,302,Manchester Utd,M,Fernandes,10.5,22.0,3,2,137,64.0,34,6.8,1.27,1.22,1.09,2.74,1.72,8.636,8.296,7.412,18.632,11.696,54.672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,230,Leicester City,M,Gray,5.3,0.1,0,0,0,0.0,0,0.0,0.86,1.48,0.85,0.66,1.71,0.000,0.000,0.000,0.000,0.000,0.000
251,227,Leicester City,D,Amartey,4.0,1.1,0,0,19,2.4,0,0.0,0.86,1.48,0.85,0.66,1.71,0.000,0.000,0.000,0.000,0.000,0.000
558,581,Wolverhampton,M,Otasowie,4.5,0.0,0,0,0,0.0,0,0.0,0.85,0.98,0.67,0.93,0.00,0.000,0.000,0.000,0.000,0.000,0.000
375,310,Manchester Utd,D,Bailly,4.9,0.2,0,0,11,2.5,-1,-0.5,1.27,1.22,1.09,2.74,1.72,-0.635,-0.610,-0.545,-1.370,-0.860,-4.020


In [ ]:
playerdataict=playerdata.sort_values('ict_index', ascending=False)
playerdataict.head(15)

,id,team,pos,name,cost,selected_by_percent,goals_scored,assists,bps,ict_index,points,points_per_game
311,254,Liverpool,M,Salah,12.4,47.6,6,1,177,96.0,50,8.3
478,388,Tottenham,F,Kane,10.9,41.1,5,8,245,89.7,65,10.8
480,390,Tottenham,M,Son,9.5,56.0,8,2,227,86.1,69,11.5
308,251,Liverpool,M,Mané,12.0,9.5,4,2,124,64.5,39,7.8
367,302,Manchester Utd,M,Fernandes,10.5,22.0,3,2,137,64.0,34,6.8
43,37,Aston Villa,M,Grealish,7.3,28.0,3,3,139,63.0,39,7.8
201,508,Everton,M,Rodríguez,8.0,38.3,3,3,176,61.6,41,6.8
193,164,Everton,F,Calvert-Lewin,7.7,56.6,7,0,179,61.6,44,7.3
164,141,Crystal Palace,M,Zaha,7.3,24.2,5,1,126,60.3,48,8.0
278,202,Leeds Utd,F,Bamford,5.9,28.8,6,2,177,59.0,49,8.2


In [ ]:
topict=playerdataict.head(30)
topict

,id,team,pos,name,cost,selected_by_percent,goals_scored,assists,bps,ict_index,points,points_per_game
311,254,Liverpool,M,Salah,12.4,47.6,6,1,177,96.0,50,8.3
478,388,Tottenham,F,Kane,10.9,41.1,5,8,245,89.7,65,10.8
480,390,Tottenham,M,Son,9.5,56.0,8,2,227,86.1,69,11.5
308,251,Liverpool,M,Mané,12.0,9.5,4,2,124,64.5,39,7.8
367,302,Manchester Utd,M,Fernandes,10.5,22.0,3,2,137,64.0,34,6.8
43,37,Aston Villa,M,Grealish,7.3,28.0,3,3,139,63.0,39,7.8
201,508,Everton,M,Rodríguez,8.0,38.3,3,3,176,61.6,41,6.8
193,164,Everton,F,Calvert-Lewin,7.7,56.6,7,0,179,61.6,44,7.3
164,141,Crystal Palace,M,Zaha,7.3,24.2,5,1,126,60.3,48,8.0
278,202,Leeds Utd,F,Bamford,5.9,28.8,6,2,177,59.0,49,8.2


https://stackoverflow.com/questions/15910019/annotate-data-points-while-plotting-from-pandas-dataframe/15911372#15911372

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import string

In [ ]:
import plotly.express as px
df = px.data.tips()

df = topict


fig = px.scatter(df, x="ict_index", y="points_per_game", text="name", log_x=True, size_max=200, color="points_per_game")
fig.update_traces(textposition='top center')
fig.update_layout(title_text='ICT top 30 players vs PPG', title_x=0.5)
fig.show()

NameError: ignored

In [ ]:
playerdata['Points per Million'] = playerdata['points'] / playerdata['cost']
playerdatappm = playerdata
playerdatappm = playerdatappm.sort_values('Points per Million', ascending=False)
top30ppm = playerdatappm.head(50)
fig = px.scatter(top30ppm, x="selected_by_percent", y="Points per Million", text="name", log_x=True, size_max=200, color="points_per_game")
fig.update_traces(textposition='top center')
fig.update_layout(title_text='Points per Million top 50 players vs % Selected', title_x=0.5)
fig.show()

In [ ]:
playerdatatop100 = playerdata.sort_values('points', ascending=False)
playerdatatop100 = playerdata.head(100)
fig = px.scatter(top30ppm, x="cost", y="points", text="name", log_x=True, size_max=200, color="points_per_game")
fig.update_traces(textposition='top center')
fig.update_layout(title_text='Points vs cost - top 100 players', title_x=0.5)
fig.show()

In [ ]:
playerdatatop100 = playerdata.sort_values('points', ascending=False)
playerdatatop100 = playerdata.head(100)
fig = px.scatter(top30ppm, x="selected_by_percent", y="points", text="name", log_x=True, size_max=200, color="points_per_game")
fig.update_traces(textposition='top center')
fig.update_layout(title_text='Points vs selected_by_percent - top 100 players', title_x=0.5)
fig.show()

In [ ]:
attackers = playerdata[playerdata['pos'] == 'F']
fig = px.scatter(attackers, x="cost", y="points", text="name", log_x=True, size_max=200, color="points_per_game")
fig.update_traces(textposition='top center')
fig.update_layout(title_text='Points vs cost - Forwards', title_x=0.5)
fig.show()

In [ ]:
midfielders = playerdata[playerdata['pos'] == 'M']
fig = px.scatter(midfielders, x="cost", y="points", text="name", log_x=True, size_max=200, color="points_per_game")
fig.update_traces(textposition='top center')
fig.update_layout(title_text='Points vs cost - midfielders', title_x=0.5)
fig.show()

In [ ]:
defenders = playerdata[playerdata['pos'] == 'D']
fig = px.scatter(defenders, x="cost", y="points", text="name", log_x=True, size_max=200, color="points_per_game")
fig.update_traces(textposition='top center')
fig.update_layout(title_text='Points vs cost - defenders', title_x=0.5)
fig.show()

In [ ]:
goalkeepers = playerdata[playerdata['pos'] == 'G']
fig = px.scatter(goalkeepers, x="cost", y="points", text="name", log_x=True, size_max=200, color="points_per_game")
fig.update_traces(textposition='top center')
fig.update_layout(title_text='Points vs cost - goalkeepers', title_x=0.5)
fig.show()

https://www.displayr.com/how-to-add-trend-lines-in-r-using-plotly/